In [16]:
import pandas as pd
import re
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import *
from sklearn.metrics import f1_score

In [17]:
input_set = pd.read_csv('input/converted.csv', sep='\t')
input_set.rename(columns={'Oceniona przez nas' : 'label'},inplace=True)

In [18]:
training_set = input_set

In [19]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
training_set.columns

Index(['Text', 'Orginalna ocena', 'label', 'Czy zgadza sie z orginalną oceną'], dtype='object')

In [20]:
print(training_set['Text'])

0      Dla mnie faworytem do tytułu będzie Cracovia. ...
1      @anonymized_account @anonymized_account Brawo ...
2      @anonymized_account @anonymized_account Super,...
3      @anonymized_account @anonymized_account Musi. ...
4        Odrzut natychmiastowy, kwaśna mina, mam problem
5      Jaki on był fajny xdd pamiętam, że spóźniłam s...
6        @anonymized_account No nie ma u nas szczęścia 😉
7      @anonymized_account Dawno kogoś tak wrednego n...
8      @anonymized_account @anonymized_account Zaległ...
9      @anonymized_account @anonymized_account @anony...
10     @anonymized_account @anonymized_account  no ma...
11     @anonymized_account @anonymized_account Może g...
12     @anonymized_account Zostawiłam tam 3 lata temu...
13     @anonymized_account Oprawa do Krzysia M. Ps Pi...
14     @anonymized_account @anonymized_account Znowu ...
15     @anonymized_account Mówi człowiek, który chcia...
16     RT @anonymized_account @anonymized_account Mów...
17     @anonymized_account @ano

In [59]:
def clean_set(df, field):
    df[field] = df[field].str.lower()
    #removal of @anonimized_account
    df[field] = df[field].apply(lambda elem: re.sub(r'@anonymized_account', '', elem))
    #removal of url links
    df[field] = df[field].apply(lambda elem: re.sub('https?://[A-Za-z0-9./]+','',elem))
    #removal of excessive whitespace,tabs etc.
    df[field] = df[field].apply(lambda elem: re.sub('\s+', ' ', elem))

In [68]:
clean_set(training_set, 'Text')

In [79]:
test_set = training_set[:100].copy()
input_set = training_set[100:].copy()

In [80]:
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('nb', SGDClassifier()),
])

In [81]:
train_majority = input_set[input_set.label==0]
train_minority = input_set[input_set.label==1]
train_majority_downsampled = resample(train_majority, 
                                 replace=True,  
                                 n_samples=len(train_minority),   
                                 random_state=123)
train_downsampled = pd.concat([train_majority_downsampled, train_minority])
train_downsampled['label'].value_counts()

1    130
0    130
Name: label, dtype: int64

In [82]:
X_train, X_test, y_train, y_test = train_test_split(train_downsampled['Text'], train_downsampled['label'], random_state=0)

In [83]:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [91]:
X_test

470     jaka tv, tacy idole😁😁😁 a tak trochę poważniej...
103     o, myślałam, że nikt już nie używa \"mnie\" t...
474             może suko to komplement, gość jest gejem
252                          maly murzyn, silny i zdrowy
406    rt plujecie na polskę to my będziemy pluć na was.
403     😁😁😁😁😁😁😁😁 teraz to mnie pani ubawiła tym paski...
358     fajnie, teraz zamiast przemowy muzyczka żeby ...
130         napewno mam niezaktualizowanego może dlatego
387     w tym czasie był z rodzinką na wakacjach w gd...
271     pierwszy priorytet zadowolić jarkacza, drugi ...
411    teraz zarykam nos i mowie \"cała polska z was ...
169         a od kiedy komentatorzy na sky to wyrocznia?
459     ale żaden z nich nie przebiera się za chuja, ...
348     odstawiając narodowość na bok to ten skład je...
440     każdy jeden klecha zrobi dla kasy wszystko i ...
248                diego maradona - robbie williams 2:1.
472              wiem że za pochlebstwo zrobisz wszystko
429    ten od kulek gejszy z bó

In [84]:
print(f1_score(y_test, y_predict))

0.7037037037037037


In [98]:
model.predict([ "bardzo ładna pogoda dzisiaj polecam wyjsc na zewtarz", "ty chuju kutasie"])

array([0, 1])